# Main notebook for battery state estimation

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys

from importlib import reload
import plotly.graph_objects as go

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback

from tensorflow.keras.models import load_model


# import os
# import os
# print("Current Directory:", os.getcwd())  # Check current working directory
# print("Files and Folders:", os.listdir())  # List all files and folders




IS_COLAB = False

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/battery-state-estimation/battery-state-estimation/"
else:
    data_path = "~/mecc/battery-state-estimation/battery-state-estimation/"

sys.path.append(data_path)
from data_processing.lg_dataset import LgData

2025-03-19 15:19:30.358878: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 15:19:30.367040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742415570.379170  580481 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742415570.381733  580481 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 15:19:30.391493: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Config logging

In [2]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [6]:
train_names = [
    '25degC/551_LA92', 
    '25degC/551_Mixed1', 
    '25degC/551_Mixed2', 
    '25degC/551_UDDS', 
    '25degC/551_US06', 
    '25degC/552_Mixed3',

    '25degC/552_Mixed7', 
    '25degC/552_Mixed8', 
    ]
test_names = [
    '25degC/552_Mixed4', 
    '25degC/552_Mixed5', 
    '25degC/552_Mixed6', 
    ]

steps = 300

lg_data = LgData(data_path)
cycles = lg_data.get_discharge_whole_cycle(train_names, test_names, output_capacity=True)
train_x, train_y, test_x, test_y = lg_data.get_discharge_multiple_step(cycles, steps)

train_y = lg_data.keep_only_y_end(train_y, steps)
test_y = lg_data.keep_only_y_end(test_y, steps)

# Model training

In [7]:
EXPERIMENT = "lstm_soc_lg"

experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
print(experiment_name)

# Model definition


opt = tf.keras.optimizers.Adam(learning_rate=0.00001)



model = Sequential()
model.add(LSTM(256, activation='selu',
                return_sequences=True,
                input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(LSTM(256, activation='selu', return_sequences=False))
model.add(Dense(256, activation='selu'))
model.add(Dense(128, activation='selu'))
model.add(Dense(1, activation='linear'))
model.summary()

model.compile(optimizer=opt, loss='huber', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])

es = EarlyStopping(monitor='val_loss', patience=50)
mc = ModelCheckpoint(data_path + 'results/trained_model/%s_best.h5' % experiment_name, 
                             save_best_only=True, 
                             monitor='val_loss')

2025-03-19-15-40-45_lstm_soc_lg


2025-03-19 15:40:45.471106: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-19 15:40:45.471126: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: BRWN325-D10
2025-03-19 15:40:45.471129: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: BRWN325-D10
2025-03-19 15:40:45.471215: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.120.0
2025-03-19 15:40:45.471231: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.120.0
2025-03-19 15:40:45.471233: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.120.0
/home/arasul42@tntech.edu/.local/lib/python3.10/site-packages/keras/src/layers/rn

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 300, 256)       │       266,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 890,369 (3.40 MB)

 Trainable params: 890,369 (3.40 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(train_x, train_y, 
                                epochs=100, 
                                batch_size=32, 
                                verbose=1,
                                validation_split=0.2,
                                callbacks = [es, mc]
                               )

Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step - loss: 0.5217 - mae: 0.9104 - mape: 201.6742 - mse: 1.2844 - rmse: 1.0977

54/54 ━━━━━━━━━━━━━━━━━━━━ 18s 309ms/step - loss: 0.5173 - mae: 0.9052 - mape: 202.0420 - mse: 1.2727 - rmse: 1.0923 - val_loss: 0.0511 - val_mae: 0.2771 - val_mape: 91.6396 - val_mse: 0.1021 - val_rmse: 0.3195
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - loss: 0.0282 - mae: 0.1839 - mape: 42.1083 - mse: 0.0564 - rmse: 0.2328

54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 303ms/step - loss: 0.0280 - mae: 0.1830 - mape: 41.9165 - mse: 0.0559 - rmse: 0.2318 - val_loss: 0.0040 - val_mae: 0.0712 - val_mape: 15.0104 - val_mse: 0.0080 - val_rmse: 0.0894
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - loss: 0.0044 - mae: 0.0741 - mape: 16.6434 - mse: 0.0087 - rmse: 0.0933

54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 308ms/step - loss: 0.0044 - mae: 0.0740 - mape: 16.7074 - mse: 0.0087 - rmse: 0.0932 - val_loss: 0.0028 - val_mae: 0.0608 - val_mape: 16.3690 - val_mse: 0.0055 - val_rmse: 0.0742
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - loss: 0.0028 - mae: 0.0593 - mape: 21.7685 - mse: 0.0056 - rmse: 0.0749

54/54 ━━━━━━━━━━━━━━━━━━━━ 18s 327ms/step - loss: 0.0028 - mae: 0.0593 - mape: 21.7420 - mse: 0.0056 - rmse: 0.0748 - val_loss: 0.0022 - val_mae: 0.0519 - val_mape: 17.1128 - val_mse: 0.0044 - val_rmse: 0.0662
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - loss: 0.0023 - mae: 0.0529 - mape: 29.1200 - mse: 0.0046 - rmse: 0.0681

...

54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 309ms/step - loss: 8.0489e-04 - mae: 0.0305 - mape: 10.5364 - mse: 0.0016 - rmse: 0.0401 - val_loss: 9.3251e-04 - val_mae: 0.0350 - val_mape: 6.2157 - val_mse: 0.0019 - val_rmse: 0.0432
Epoch 56/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 314ms/step - loss: 9.4265e-04 - mae: 0.0324 - mape: 9.4753 - mse: 0.0019 - rmse: 0.0434 - val_loss: 0.0018 - val_mae: 0.0481 - val_mape: 8.6543 - val_mse: 0.0035 - val_rmse: 0.0592
Epoch 57/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 307ms/step - loss: 9.7915e-04 - mae: 0.0340 - mape: 7.6164 - mse: 0.0020 - rmse: 0.0442 - val_loss: 0.0034 - val_mae: 0.0679 - val_mape: 8.8264 - val_mse: 0.0069 - val_rmse: 0.0828
Epoch 58/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 311ms/step - loss: 0.0012 - mae: 0.0371 - mape: 8.6771 - mse: 0.0024 - rmse: 0.0485 - val_loss: 9.7370e-04 - val_mae: 0.0361 - val_mape: 7.1190 - val_mse: 0.0019 - val_rmse: 0.0441
Epoch 59/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 309ms/step - loss: 8.7804e-04 - mae: 0.0315 - mape: 7.1391 - mse

54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 314ms/step - loss: 9.2429e-04 - mae: 0.0330 - mape: 5.9456 - mse: 0.0018 - rmse: 0.0429 - val_loss: 9.1007e-04 - val_mae: 0.0349 - val_mape: 6.1639 - val_mse: 0.0018 - val_rmse: 0.0427
Epoch 72/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 311ms/step - loss: 7.6046e-04 - mae: 0.0299 - mape: 7.9764 - mse: 0.0015 - rmse: 0.0390 - val_loss: 0.0012 - val_mae: 0.0393 - val_mape: 6.2928 - val_mse: 0.0023 - val_rmse: 0.0482
Epoch 73/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 311ms/step - loss: 7.1824e-04 - mae: 0.0285 - mape: 6.6248 - mse: 0.0014 - rmse: 0.0379 - val_loss: 0.0015 - val_mae: 0.0460 - val_mape: 7.2255 - val_mse: 0.0031 - val_rmse: 0.0557
Epoch 74/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 307ms/step - loss: 7.7442e-04 - mae: 0.0301 - mape: 10.4242 - mse: 0.0015 - rmse: 0.0393 - val_loss: 0.0012 - val_mae: 0.0413 - val_mape: 6.7712 - val_mse: 0.0025 - val_rmse: 0.0499
Epoch 75/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step - loss: 7.8540e-04 - mae: 0.0304 - mape: 14.1372 - mse

54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 310ms/step - loss: 7.8721e-04 - mae: 0.0304 - mape: 14.0564 - mse: 0.0016 - rmse: 0.0396 - val_loss: 8.7401e-04 - val_mae: 0.0332 - val_mape: 6.7144 - val_mse: 0.0017 - val_rmse: 0.0418
Epoch 76/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 314ms/step - loss: 0.0014 - mae: 0.0405 - mape: 7.2085 - mse: 0.0028 - rmse: 0.0530 - val_loss: 0.0015 - val_mae: 0.0450 - val_mape: 5.9865 - val_mse: 0.0031 - val_rmse: 0.0555
Epoch 77/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 322ms/step - loss: 7.9830e-04 - mae: 0.0304 - mape: 6.0502 - mse: 0.0016 - rmse: 0.0399 - val_loss: 0.0014 - val_mae: 0.0433 - val_mape: 5.9282 - val_mse: 0.0028 - val_rmse: 0.0533
Epoch 78/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 313ms/step - loss: 8.7722e-04 - mae: 0.0318 - mape: 5.5397 - mse: 0.0018 - rmse: 0.0418 - val_loss: 0.0015 - val_mae: 0.0454 - val_mape: 6.1324 - val_mse: 0.0030 - val_rmse: 0.0547
Epoch 79/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 309ms/step - loss: 7.4356e-04 - mae: 0.0294 - mape: 4.6874 - mse: 0.

54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 305ms/step - loss: 9.7399e-04 - mae: 0.0345 - mape: 8.7616 - mse: 0.0019 - rmse: 0.0441 - val_loss: 8.3609e-04 - val_mae: 0.0330 - val_mape: 4.3261 - val_mse: 0.0017 - val_rmse: 0.0409
Epoch 91/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 307ms/step - loss: 7.9179e-04 - mae: 0.0301 - mape: 5.3857 - mse: 0.0016 - rmse: 0.0397 - val_loss: 0.0021 - val_mae: 0.0540 - val_mape: 7.8441 - val_mse: 0.0042 - val_rmse: 0.0650
Epoch 92/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 305ms/step - loss: 9.9247e-04 - mae: 0.0343 - mape: 6.4105 - mse: 0.0020 - rmse: 0.0445 - val_loss: 8.5588e-04 - val_mae: 0.0347 - val_mape: 7.5392 - val_mse: 0.0017 - val_rmse: 0.0414
Epoch 93/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 309ms/step - loss: 7.4528e-04 - mae: 0.0294 - mape: 5.0395 - mse: 0.0015 - rmse: 0.0386 - val_loss: 8.5187e-04 - val_mae: 0.0338 - val_mape: 5.0889 - val_mse: 0.0017 - val_rmse: 0.0413
Epoch 94/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 306ms/step - loss: 6.9653e-04 - mae: 0.0283 - mape: 6.526

54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 304ms/step - loss: 9.0415e-04 - mae: 0.0329 - mape: 7.7781 - mse: 0.0018 - rmse: 0.0423 - val_loss: 8.0503e-04 - val_mae: 0.0327 - val_mape: 6.8706 - val_mse: 0.0016 - val_rmse: 0.0401
Epoch 99/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 306ms/step - loss: 6.8771e-04 - mae: 0.0286 - mape: 5.5957 - mse: 0.0014 - rmse: 0.0371 - val_loss: 0.0010 - val_mae: 0.0349 - val_mape: 6.0659 - val_mse: 0.0020 - val_rmse: 0.0448
Epoch 100/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 310ms/step - loss: 8.1973e-04 - mae: 0.0308 - mape: 6.8194 - mse: 0.0016 - rmse: 0.0405 - val_loss: 0.0010 - val_mae: 0.0369 - val_mape: 5.1961 - val_mse: 0.0020 - val_rmse: 0.0451


In [7]:
model.save(data_path + 'results/trained_model/%s.h5' % experiment_name)

hist_df = pd.DataFrame(history.history)
hist_csv_file = data_path + 'results/trained_model/%s_history.csv' % experiment_name
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


### Testing

In [9]:
model=load_model('/home/arasul42@tntech.edu/mecc/battery-state-estimation/battery-state-estimation/experiments/lg/~/mecc/battery-state-estimation/battery-state-estimation/results/trained_model/2025-03-18-23-14-33_lstm_soc_lg_best.h5') 
results = model.evaluate(test_x, test_y)
print(results)

2025-03-19 00:01:10.198707: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-19 00:01:10.198729: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: BRWN325-D10
2025-03-19 00:01:10.198732: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: BRWN325-D10
2025-03-19 00:01:10.198814: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.120.0
2025-03-19 00:01:10.198831: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.120.0
2025-03-19 00:01:10.198833: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.120.0


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step - loss: 340455.6250 - mean_absolute_error: 340456.1250 - mean_absolute_percentage_error: 29162854.0000 - mean_squared_error: 28147055591424.0000 - rmse: 4506724.5000
[332638.96875, 28628716879872.0, 332639.46875, 28198410.0, 5350581.0]


# Data Visualization

In [9]:
import nbformat
print(nbformat.__version__)

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'],
                    mode='lines', name='train'))
fig.add_trace(go.Scatter(y=history.history['val_loss'],
                    mode='lines', name='validation'))
fig.update_layout(title='Loss trend',
                  xaxis_title='epoch',
                  yaxis_title='loss',
                  width=1400,
                  height=600)
fig.write_image(data_path + 'results/figures/%s_loss.png' % experiment_name)

5.10.4


FileNotFoundError: [Errno 2] No such file or directory: '~/mecc/battery-state-estimation/battery-state-estimation/results/figures/2025-03-19-15-40-45_lstm_soc_lg_loss.png'

In [11]:
train_predictions = model.predict(train_x)

67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 112ms/step


In [12]:
cycle_num = 0
steps_num = 8000
step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

fig = go.Figure()
fig.add_trace(go.Scatter(x=step_index, y=train_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC predicted'))
fig.add_trace(go.Scatter(x=step_index, y=train_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC actual'))
fig.update_layout(title='Results on training',
                  xaxis_title='Step',
                  yaxis_title='SoC capacity',
                  width=1400,
                  height=600)
fig.show()

In [13]:
test_predictions = model.predict(test_x)

23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step


In [15]:
cycle_num = 0
steps_num = 8000
step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

fig = go.Figure()
fig.add_trace(go.Scatter(x=step_index, y=test_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC predicted'))
fig.add_trace(go.Scatter(x=step_index, y=test_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
                    mode='lines', name='SoC actual'))
fig.update_layout(title='Results on testing',
                  xaxis_title='Step',
                  yaxis_title='SoC capacity',
                  width=1400,
                  height=600)
fig.show()